In [1]:
from discussion_agents.cog.agent.reflexion import ReflexionReActAgent

In [2]:
from discussion_agents.cog.agent.reflexion import ReflexionReActAgent
from langchain_community.chat_models import ChatOpenAI

import dotenv
import os

dotenv.load_dotenv("../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(openai_api_key=openai_api_key)

agent = ReflexionReActAgent(
    self_reflect_llm=llm,
    action_llm=llm,
    max_steps=7,
    max_tries=3,
)

In [3]:
agent

In [4]:
question = "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"
key = "Gesellschaft mit beschränkter Haftung"

In [5]:
out = agent.generate(question=question, key=key, strategy="reflexion")

TRIAL 0


In [9]:
len(out)

1

In [10]:
print(out[0])


Thought: I need to search for VIVA Media AG and find out what their new acronym stands for.
Action: Search[VIVA Media AG]
Observation 1: Could not find [VIVA Media AG]. Similar: ['MTV Music (Polish TV channel)', 'Paramount International Networks', 'VIVA Plus', 'VIVA (German TV channel)', 'Viacom (1952–2005)', 'Vauxhall Viva', 'GfK Entertainment charts', 'Lindt', 'Spellbound Entertainment', 'Ag-gag']
Thought: The search for "VIVA Media AG" did not yield any results. I should try searching for their new acronym instead.
Action: Search[new acronym VIVA Media AG]
Observation 2: Could not find [new acronym VIVA Media AG]. Similar: ['Holden Torana', 'Earth Liberation Front', 'Andrés Manuel López Obrador']
Thought: The search for the new acronym of VIVA Media AG also did not yield any results. I should try searching for any name changes or rebranding of the company in 2004.
Action: Search[VIVA Media AG name change 2004]
Observation 3: Could not find [VIVA Media AG name change 2004]. Similar:

In [7]:
agent.reflector.reflections

[]